In [1]:
from Model import Model
import pandas as pd
import numpy as np
import itertools
import shap
from sklearn.model_selection import train_test_split
from Parameters import Parameters

t = Parameters('DT').training_reg_params

%load_ext autoreload
%autoreload 2

In [2]:
model = Model('DT')

In [3]:
# Y = pd.read_excel('../../Frame_Analysis/Microsoft Ad Optimization-AI Data_Survey Aggregated Data 21062019.xlsx',sheet_name="Sheet2")
# ub = Y[['Video Name','Unaided Branding']].groupby(['Video Name','Unaided Branding'])
# Y_ub = ub.first()

In [4]:
X,Y = model.preprocessing.load_data('../Agg_data_from_frame_163.xlsx',labels=False,sheet_name="Sheet1")
Y_ub = model.preprocessing.load_Y('./Agg_Unaided.xlsx',video_name=False,column_name="same")
Y_in = model.preprocessing.load_Y('../Updated_targets_300519.xlsx',video_name=False,column_name="diff")

In [5]:
Y = pd.DataFrame(index=Y_ub.index.values,columns=['Unaided_Branding'],data=Y_ub.values)
# Y['Unaided_Branding'] = Y_ub
Y.loc[Y.index.values,'Active_Involvement_Mean'] = Y_in.loc[Y.index.values,['Active_Involvement__Mean']].values

In [6]:
X = X.set_index(X['Video Name'])
X = X.drop('Unnamed: 0',1)
# Y_old['Video Name'] = Y_old.index.values
# Y_new['Video Name'] = Y_new.index.values

# Y_new = Y_new[Y_new['Video Name']!= 'Surface Laptop_UK_30_Courtney v2 US VO_TV']
# Y_new = Y_new[Y_new['Video Name']!='284 Surface Laptop_UK_30_Courtney v2 US VO_TV' ]
# Y_new = Y_new.drop('Video Name',1)
# Y = pd.concat((Y_old,Y_new),axis=1)
X = X.loc[X['Campaign']=='Software']
Y =  Y.rename(str,{'Unaided Branding':'Unaided_Branding'})
Y = Y.loc[X.index.values,:]

In [7]:
Y.dropna(how='all')
# X = X.drop('Song for Surface NFL Fans - Surface SP4 30 TV US',0)
X,Y= model.preprocessing.clean_data(X,Y)

In [8]:
surface_pro_frames = [col for col in X.columns if "Surface Pro".lower() in col.lower()]
# for suf in ['_dur','_first_ocr','_last_ocr','_total_ocr']:
#     cols = [col+suf for col in surface_pro_frames[:10]]
#     X = model.preprocessing.addNewFeature(X,cols,_type="sum",new_column_name="Surface"+suf)
# X = X.drop(surface_pro_frames,axis=1)

In [9]:
frame_count_columns = [col for col in X.columns.values if '_dur' in col]
columns_removed =[]
for x in X.columns.values:
    if '_frame_count' in x:
        columns_removed.append(x)
cols = [col[:-12] for col in columns_removed]
for x in frame_count_columns:
#     if x+'_dur' in X.columns.values or '_frame_count' in x:
    if x[:-4] not in cols:
        columns_removed.append(x[:-4])
columns_removed.remove('Intel')
# columns_removed.remove('Surface_group')

In [10]:
X = X.drop(columns=columns_removed,axis=1)
columns_removed

['African_frame_count',
 'Caucasian_frame_count',
 'Eastasian_frame_count',
 'Latino_frame_count',
 'Angry_frame_count',
 'Disgust_frame_count',
 'Fear_frame_count',
 'Happiness_frame_count',
 'Neutral_frame_count',
 'Sadness_frame_count',
 'Surprise_frame_count',
 'Age: 0-15_frame_count',
 'Age: 15-24_frame_count',
 'Age: 25-34_frame_count',
 'Age: 35-44_frame_count',
 'Age: 45-60_frame_count',
 'Female_frame_count',
 'Male_frame_count',
 'microsoft',
 'ASUS T102',
 'Dell_sub_brand_group',
 'HP_group',
 'Lenovo_group',
 'Macbook_group',
 'Surface_group',
 'Windows',
 'Yoga',
 'Asus_text',
 'Currys PCs world_text',
 'Dell_text',
 'Hp_text',
 'Lenovo_text',
 'Apple',
 'Asus',
 'Dell',
 'Hp',
 'Microsoft_text',
 'Laptop',
 'Tablet',
 'Mobile',
 'PC']

In [11]:
new_features =[["Happiness_dur",'Female_dur'],["Surprise_dur","Male_dur"],["Neutral_dur","Male_dur"],["Happiness_dur","Age: 0-5_dur"],["Disgust_dur","Age: 45-60_dur"],["Surprise_dur","Age: 0-5_dur"],["Neutral_dur","Age: 25-34_dur"],["Happiness_dur","Eastasian_dur"],["Surprise_dur","Eastasian_dur"]]
age_cols = ['Age: 0-15_dur', 'Age: 15-24_dur','Age: 25-34_dur', 'Age: 35-44_dur', 'Age: 45-60_dur']
ethinicity_cols = ['African_dur', 'Caucasian_dur', 'Eastasian_dur','Latino_dur']
gender_cols = ["Female_dur","Male_dur"]
emotion_cols = ["Happiness_dur","Surprise_dur","Neutral_dur","Sadness_dur"]
# age_ethinicity_col = list(itertools.product(age_cols,ethinicity_cols))
# new_features.extend(age_ethinicity_col)
# X = model.preprocessing.addNewFeature(X,new_features,_type='per')
# X['Happiness_length_of_ad_impact'] = X['Happiness_dur'] * (1-X['length_of_ad_15'])

In [12]:
X = X.drop(age_cols,1)
X = X.drop(ethinicity_cols,1)
X = X.drop(gender_cols,1)
X = X.drop(emotion_cols,1)

In [13]:
model.preprocessing.checkNull(X).head()
X= model.preprocessing.removeNULLColumns(X,percentage=0.8)
model.preprocessing.checkNull(X).head()

Tablet_total_ocr     79.545455
Tablet_first_ocr     79.545455
Tablet_dur           79.545455
Tablet_last_ocr      79.545455
Hp_text_total_ocr    72.727273
dtype: float64

In [14]:
ocr_columns = [col for col in X.columns if "ocr" in col]

X[ocr_columns] = model.preprocessing.imputeConstant(X,columns=ocr_columns,constant=-1)
X = model.preprocessing.replace(X,np.inf,np.nan)

In [15]:
X = model.preprocessing.imputeConstant(X,constant=0)
X = X.fillna(0)

In [16]:
def bin_first_ocr(x):
    df = pd.DataFrame(index=[0],columns=x.index)
#     print(df)
    for i,col in enumerate(x):
        if col ==-1:
            df.loc[0,x.index[i]] = -1
        elif col <=1:
            df.loc[0,x.index[i]] = 0
        elif col<=6.5:
            df.loc[0,x.index[i]] = 1
        elif col<=14:
            df.loc[0,x.index[i]] =2
        else:
            df.loc[0,x.index[i]] =3
    return df

def bin_dur(x):
    df = pd.DataFrame(index=[0],columns=x.index)
    for i,col in enumerate(x):
        if col <=4:
            df.loc[0,x.index[i]]=0
        elif col <=5.5:
            df.loc[0,x.index[i]] =1
        elif col<=7:
            df.loc[0,x.index[i]] =2 
        else:
            df.loc[0,x.index[i]] = 3
    return df

def bin_last_ocr(x):
    df = pd.DataFrame(index=[0],columns=x.index)
    for i,col in enumerate(x):
        if col == -1:
            df.loc[0,x.index[i]] =-1
        elif col <= 15:
            df.loc[0,x.index[i]] = 0
        elif col<=24:
            df.loc[0,x.index[i]] = 1
        else:
            df.loc[0,x.index[i]]  = 2
    return df

In [17]:
first_ocr = [col for col in X.columns if 'first_ocr' in col]
last_ocr =  [col for col in X.columns if 'last_ocr' in col]
dur = [col for col in X.columns if '_dur' in col]

In [18]:
f = X[first_ocr].apply(bin_first_ocr,1)
l = X[last_ocr].apply(bin_last_ocr,1)
d = X[dur].apply(bin_dur,1)
for i,ind in enumerate(f.index.values):
#     f[0] = f[0].set_index(ind)
    for j,col in enumerate(first_ocr):
        X.loc[ind,col] = f[i].values[0][j]
    for j,col in enumerate(last_ocr):
        X.loc[ind,col] = l[i].values[0][j]
    
    for j,col in enumerate(dur):
        X.loc[ind,col] = d[i].values[0][j]    


In [19]:
from sklearn.preprocessing import OneHotEncoder
ohe_first_cols = OneHotEncoder(handle_unknown="ignore",categories=[[-1,0,1,2,3]]*len(first_ocr))
ohe_last_cols = OneHotEncoder(handle_unknown="ignore",categories=[[-1,0,1,2]]*len(last_ocr))
ohe_dur_cols = OneHotEncoder(handle_unknown="ignore",categories=[[0,1,2,3]]*len(dur))

In [20]:
f_ohe = ohe_first_cols.fit_transform(X[first_ocr]).toarray()
l_ohe = ohe_last_cols.fit_transform(X[last_ocr]).toarray()
d_ohe = ohe_dur_cols.fit_transform(X[dur]).toarray()

In [21]:
first_cols = ["_no_ocr","_0_1","_1.5_6.5","_7_14","_above_14"]
last_cols = ["_0_4","_4.5_5.5","_6_7","_above_7.5"]
dur_cols = ["_no_ocr","_0_14","_15_24","_above_25"]
first_ohe,last_ohe,dur_ohe=[],[],[]
for prefix in first_ocr:
    for suffix in first_cols:
        first_ohe.append(prefix+suffix)
        
for prefix in last_ocr:
    for suffix in last_cols:
        last_ohe.append(prefix+suffix)
        
for prefix in dur:
    for suffix in dur_cols:
        dur_ohe.append(prefix+suffix)

In [22]:
ohe_df =pd.DataFrame(columns=first_ohe,data=f_ohe,index=X[first_ocr].index)
ohe_df = pd.concat((ohe_df,pd.DataFrame(columns=last_ohe,data=l_ohe,index=X[last_ocr].index)),axis=1)
ohe_df = pd.concat((ohe_df,pd.DataFrame(columns=dur_ohe,data=d_ohe,index=X[dur].index)),axis=1)

In [23]:
X = pd.concat((X,ohe_df),1)
X = X.drop(first_ocr,1)
X = X.drop(last_ocr,1)
X = X.drop(dur,1)

In [24]:
X.shape

(44, 142)

In [25]:
# Y = model.preprocessing.imputeMedian(Y,model.preprocessing.targets)
Y.isna().sum()

Unaided_Branding           0
Active_Involvement_Mean    0
dtype: int64

In [26]:
# Y_14 = Y.loc[:,Y.isna().sum()==0]
# Y_4 = Y.loc[:,Y.isna().sum()!=0]
# X_4 = X[Y_4['Unaided_Branding'].isna()==False]
# Y_4 = Y_4[Y_4['Unaided_Branding'].isna()==False]

In [27]:
X_res_1,Y_res_1 = model.preprocessing.overSampling(X,Y,each_Col=False,size=1000,random_state=12)
X_train_1,X_test_1,y_train_1,y_test_1= train_test_split(X_res_1,Y_res_1,random_state = 123,test_size=0.3)

# X_res_4,Y_res_4 = model.preprocessing.overSampling(X_4,Y_4,each_Col=False,size=1000,random_state=12)
# X_train_4,X_test_4,y_train_4,y_test_4 = train_test_split(X_res_4,Y_res_4,random_state = 123,test_size=0.3)

In [28]:
# print("------------------------ 14 Models Training ----------------------------\n")
# models_14 = model.train(X_train_14,y_train_14,X_test_14,y_test_14,metrics='r2',plot_error=False,set_params=True,params=t)
print("------------------------ 1 Models Training ----------------------------\n")
models_1 = model.train(X_train_1,y_train_1,X_test_1,y_test_1,metrics='r2',plot_error=False,set_params=True,params={'Unaided_Branding': {'splitter':'best','max_depth': 10, 'max_features': 0.6, 'min_samples_leaf': 15,'min_samples_split':15,'presort':True},'Active_Involvement_Mean':{'max_depth': 10, 'max_features': 0.8, 'min_samples_leaf': 15,'presort':True}})

------------------------ 1 Models Training ----------------------------

Unaided_Branding


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:    1.1s remaining:    1.6s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    1.3s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


For training set
r2_score: 0.833011
For test set
r2_score: 0.909502


Active_Involvement_Mean


[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:    0.0s remaining:    0.1s


For training set
r2_score: 0.665789
For test set
r2_score: 0.718768




[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.6s finished


In [29]:
models_1[0]sq

SyntaxError: invalid syntax (<ipython-input-29-752f3c6c245f>, line 1)

In [ ]:
model.plot_tree(savefig=True)

In [ ]:
model.plot_feature_importance(models_1,10,y_train_1.columns,X_train_1.columns,savefig=False)

In [ ]:
model_xgb = Model('xgb')
print("------------------------ XGB Unaided Model Training ----------------------------\n")
models_xgb_1 = model_xgb.train(X_train_1,y_train_1,X_test_1,y_test_1,metrics='r2',plot_error=False)

In [ ]:
model_xgb.plot_feature_importance(models_xgb_1,10,y_train_1.columns,X_train_1.columns,savefig=False)

In [ ]:
from tqdm import tqdm
shap.initjs()
explainer=[]
shap_values=[]
for i in tqdm(range(1)):
    explainer.append(shap.TreeExplainer(models_1[i]))
    shap_values.append(explainer[-1].shap_values(X_train_1))
    
    print(i)

In [ ]:
req_cols = [
'microsoft_first_ocr',
'Laptop_first_ocr',
'microsoft_last_ocr',
'Happiness_dur',
'Laptop_last_ocr',
'Laptop_dur',
'Neutral_Male_dur',
'Caucasian_dur',
'length_of_ad_15',
'Happiness_Female_dur',
'Female_dur'
]

In [ ]:

model.shap_analysis(shap_values,X_train_1,y_train_1.columns,X_train_1.columns,n_features=11,required_cols=req_cols)

In [ ]:
import eli5
from eli5.sklearn import PermutationImportance

perm = PermutationImportance(models_1[0], random_state=1).fit(X_train_1, y_train_1)
eli5.show_weights(perm, feature_names = X_train_1.columns.tolist())

In [ ]:
col = 'Caucasian_dur'
X_train_1[col].mean()

In [ ]:
a = np.array([1,2,3,4,5])

In [ ]:
a.std()

In [ ]:
shap.summary_plot(shap_values[0],X_train_1)

In [ ]:
X_train_14.to_excel('../../Version_1.01/x_train_14.xlsx')

In [ ]:
X_test_14.to_excel('../../Version_1.01/x_test_14.xlsx')
y_train_14.to_excel('../../Version_1.01/y_train_14.xlsx')
y_test_14.to_excel('../../Version_1.01/y_test_14.xlsx')

In [ ]:
X_train_4.to_excel('../../Version_1.01/x_train_4.xlsx')
X_test_4.to_excel('../../Version_1.01/x_test_4.xlsx')
y_train_4.to_excel('../../Version_1.01/y_train_4.xlsx')
y_test_4.to_excel('../../Version_1.01/y_test_4.xlsx')